<b> ZKOUSKA prikazu podle webu </b>

In [72]:
import psycopg2
import pandas as pd

print('Manual connecting to the PostgreSQL database...\n\n\n')
conn = psycopg2.connect(
    host="postgres",
    database="dvdrental",
    user="postgres",
    password="example")

def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

# create a cursor
cur = conn.cursor()

#LIST TABULEK BASE TABLE
tab_info = create_pandas_table("SELECT table_name FROM information_schema.tables WHERE table_type = 'BASE TABLE' AND table_schema NOT IN ('pg_catalog', 'information_schema');")

print('Tabulky v databazi:\n')
print(tab_info)

#LIST TABULEK VIEWs
view_info = create_pandas_table("SELECT table_name FROM information_schema.tables WHERE table_type = 'VIEW' AND table_schema NOT IN ('pg_catalog', 'information_schema') AND table_name !~ '^pg_';")
print('\n\n\nViews v aktualni databazi: \n')
print(view_info)

#LIST USERS
list_users = create_pandas_table("SELECT usename FROM pg_user;")
print('\n\n\nUzivatele v aktualni databazi: \n')
print(list_users)

#RADKY v databazi
colum_info = create_pandas_table("SELECT column_name FROM information_schema.columns WHERE table_name = 'store';")

print('\n\n\nINFO O tabulce: *JMENOTABULKY* \n')
print(colum_info)

#Detailni info o radcich
detail_info = create_pandas_table("SELECT ordinal_position, column_name, data_type, column_default, is_nullable, character_maximum_length, numeric_precision FROM information_schema.columns WHERE table_name = 'store' ORDER BY ordinal_position;")

print('\n\n Detailní info o tabulce : *JMENOTABULKY* \n')
print(detail_info)

#LIST TRIGGERS#
trigger_info = create_pandas_table("SELECT DISTINCT trigger_name FROM information_schema.triggers WHERE trigger_schema NOT IN ('pg_catalog', 'information_schema');")

print('\n\n List of triggrs : \n')
print(trigger_info)

#Detailni LIST TRIGGERS (pro vsechno v databazi)#
#Detail_trigger_info = create_pandas_table("SELECT * FROM information_schema.triggers WHERE trigger_schema NOT IN ('pg_catalog', 'information_schema');")
#print('\n\n Detailni List of triggrs : \n')
#print(Detail_trigger_info)

# close the communication with the PostgreSQL
cur.close()
conn.close()

Manual connecting to the PostgreSQL database...



Tabulky v databazi:

       table_name
0           actor
1           store
2         address
3        category
4            city
5         country
6        customer
7      film_actor
8   film_category
9       inventory
10       language
11         rental
12          staff
13        payment
14           film



Views v aktualni databazi: 

                   table_name
0                  actor_info
1               customer_list
2                   film_list
3  nicer_but_slower_film_list
4      sales_by_film_category
5              sales_by_store
6                  staff_list



Uzivatele v aktualni databazi: 

    usename
0  postgres



INFO O tabulce: *JMENOTABULKY* 

        column_name
0          store_id
1  manager_staff_id
2        address_id
3       last_update


 Detailní info o tabulce : *JMENOTABULKY* 

   ordinal_position       column_name                    data_type  \
0                 1          store_id                   

<b>pomocí SQL prikazu dotaz jak vypada schema a tabulka</b>

In [89]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(
    host="postgres",
    database="postgres",
    user="postgres",
    password="example")

#kurzor na cteni/zapis z databaze pomoci pythonu
cur = conn.cursor()

def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table


#SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';

schemata = create_pandas_table("SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';")

print('Schemata v databazi:\n')
print(schemata)

table_schema = create_pandas_table("SELECT * FROM information_schema.columns WHERE TABLE_NAME = 'uzivatel';")
 

print('\n\nSchemata v tabulce:\n')
print(table_schema)

shZK = create_pandas_table("SELECT schemaname FROM pg_catalog.pg_tables WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';")
print('\n\n SCHEMATA?:\n')
print(shZK)


cur.close()
conn.close()

Schemata v databazi:

  schemaname   tablename tableowner tablespace  hasindexes  hasrules  \
0     public       users   postgres       None        True     False   
1     public  usergroups   postgres       None        True     False   
2     public      groups   postgres       None        True     False   
3     public    uzivatel   postgres       None        True     False   

   hastriggers  rowsecurity  
0         True        False  
1         True        False  
2         True        False  
3        False        False  


Schemata v tabulce:

  table_catalog table_schema table_name     column_name  ordinal_position  \
0      postgres       public   uzivatel    uzivatele_id                 1   
1      postgres       public   uzivatel           jmeno                 2   
2      postgres       public   uzivatel        prijmeni                 3   
3      postgres       public   uzivatel  datum_narozeni                 4   
4      postgres       public   uzivatel    pocet_clanku    

In [26]:
pip install flask

     |████████████████████████████████| 94 kB 2.2 MB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


<h1><b> ZÍSKÁNÍ struktury z databaze BETA </b></h1>

In [161]:
import psycopg2
import flask

#------------Pripojeni-k-databazi---------------------
conn = psycopg2.connect(
    host="postgres",
    database="postgres",
    user="postgres",
    password="example")

#------------Vytvoreni-kurzoru-------------------
db_cursor = conn.cursor()

#---------------------Najiti-vsech-schemat-v-databazi-------------------------------------
def GetSchema():
    p = "SELECT schemaname FROM pg_catalog.pg_tables WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';" #ignorovani schemat catalog a information
    
    db_cursor.execute(p)
    list_schemat = db_cursor.fetchall()
    
    list_schemat = list(dict.fromkeys(list_schemat)) #odtraneni duplicitnich schemat v listu a vytvoreni noveho kde je kazde schema jednou !
    
    return(list_schemat)

#------------------Najiti-tabulek-v-kazdem-schematu-v-databazi--------------------------------------
def GetTableList(schema):
    s = ""
    s += "SELECT"
    s += " table_name"
    s += " FROM information_schema.tables"
    s += " WHERE"
    s += " ("
    s += " table_schema = '" + schema + "'"
    s += " )"
    #s += " ORDER BY table_schema, table_name;"

    # Retrieve all the rows from the cursor
    db_cursor.execute(s)
    list_tables = db_cursor.fetchall()

    # Print the names of the tables
    return(list_tables)
    #for t_name_table in list_tables:
        #print(t_name_table)
     #   return(t_name_table) # POUZE jmeno tabulky
    
#----------------------Popis-radku-v-tabulkach-------------------------------------
def GetColInfo(radek):
    l = "SELECT"
    l += " table_name, ordinal_position, column_name, data_type, column_default, is_nullable, character_maximum_length, numeric_precision"
    l += " FROM information_schema.columns"
    l += " WHERE TABLE_NAME = '" + radek + "';"
    
    db_cursor.execute(l)
    list_colums = db_cursor.fetchall()
    
    #print(list_colums)
    return(list_colums)
    #for nam in list_colums:
    #    print(nam)
        
#---------------------Spusteni-hlednai-v-databazi-a-ukladani-do-TEXTAKU-----------------------------------       
f = open("SQL_STRUCT.txt","w+")

TestSchema = GetSchema()
for sch in TestSchema:
    #print(sch[0])
    schema = sch[0]
    f.write("CREATE SCHEMA "+sch[0]+"\n")
    seznam_tabulek = GetTableList(schema)
    for tabulka in seznam_tabulek:
        #print(tabulka[0])
        f.write("\tCREATE TABLE "+sch[0]+"."+tabulka[0]+"(\n")
        Column_Info = GetColInfo(tabulka[0])
        #GetColInfo(tabulka[0])
        i=0
        for column in Column_Info:
            print(column)
            i=i+1
            if i!= len(Column_Info):
                f.write("\t\t"+column[2]+" "+column[3]+",\n")
            else:
                f.write("\t\t"+column[2]+" "+column[3]+"\n\t)\n")     #PROVIZORNE ")" POTOM DODELAT DALSI PODMINKY NA PRIMARNI KLIC A JINE PRVKY KTERE SE MOHOU VYSKYTOVAT V DATABAZI
            #print(column[2])
            #print(len(Column_Info))
        
        
#schema = "public"
#seznam_tabulek = GetTableList(schema)
#for tabulka in seznam_tabulek:
#    GetColInfo(tabulka[0])
    #print(s[0])
#print(seznam_tabulek)
#rad = "uzivatel"
#GetColInfo(rad)



#---------!! UKONCENI SPOJENI !!-----------
db_cursor.close()
conn.close()

f.close()

('users', 1, 'id', 'bigint', None, 'NO', None, 64)
('users', 2, 'name', 'character varying', None, 'YES', None, None)
('usergroups', 1, 'id', 'bigint', None, 'NO', None, 64)
('usergroups', 2, 'user_id', 'bigint', None, 'YES', None, 64)
('usergroups', 3, 'group_id', 'bigint', None, 'YES', None, 64)
('groups', 1, 'id', 'bigint', None, 'NO', None, 64)
('groups', 2, 'name', 'character varying', None, 'YES', None, None)
('uzivatel', 1, 'uzivatele_id', 'integer', "nextval('uzivatel_uzivatele_id_seq'::regclass)", 'NO', None, 32)
('uzivatel', 2, 'jmeno', 'character varying', None, 'YES', 60, None)
('uzivatel', 3, 'prijmeni', 'character varying', None, 'YES', 60, None)
('uzivatel', 4, 'datum_narozeni', 'date', None, 'YES', None, None)
('uzivatel', 5, 'pocet_clanku', 'integer', None, 'YES', None, 32)
